# Single Frame Model

This script trains a single frame model based on an average image taken across the the forcasted month.

In [72]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, Conv3D, MaxPooling2D, MaxPooling3D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam
import time
import numpy as np

## 1. Load Data

To begin, data for a single frame mnodel must be loaded along with the expected regional rainfall values.

In [25]:
training_datafile = "E:/31-12-2020/prepared-data/single_train.npy"
validation_datafile = "E:/31-12-2020/prepared-data/single_valid.npy"

training_rainfallfile = "E:/31-12-2020/prepared-data/expected_train.npy"
validation_rainfallfile = "E:/31-12-2020/prepared-data/expected_valid.npy"

In [26]:
training_videos = np.load(training_datafile)
validation_videos = np.load(validation_datafile)

training_rainfall = np.load(training_rainfallfile)[:, 2:]
validation_rainfall = np.load(validation_rainfallfile)[:, 2:]

In [27]:
# We need them in X, Y, COLOURS
training_videos = np.swapaxes(training_videos, 1, 2)
training_videos = np.swapaxes(training_videos, 2, 3)

validation_videos = np.swapaxes(validation_videos, 1, 2)
validation_videos = np.swapaxes(validation_videos, 2, 3)

In [28]:
# Scale the data
training_videos[:, :, :, 0] = (training_videos[:, :, :, 0] - np.min(training_videos[:, :, :, 0])) / (np.max(training_videos[:, :, :, 0]) - np.min(training_videos[:, :, :, 0]))
training_videos[:, :, :, 1] = (training_videos[:, :, :, 1] - np.min(training_videos[:, :, :, 1])) / (np.max(training_videos[:, :, :, 1]) - np.min(training_videos[:, :, :, 1]))

validation_videos[:, :, :, 0] = (validation_videos[:, :, :, 0] - np.min(validation_videos[:, :, :, 0])) / (np.max(validation_videos[:, :, :, 0]) - np.min(validation_videos[:, :, :, 0]))
validation_videos[:, :, :, 1] = (validation_videos[:, :, :, 1] - np.min(validation_videos[:, :, :, 1])) / (np.max(validation_videos[:, :, :, 1]) - np.min(validation_videos[:, :, :, 1]))

## 2. Model Definition

Next, a CNN model architecture is defined.

In [111]:
def model_generator(input_shape=(2, 61, 121), learning_rate=0.1):
    """ This method generates a model definition. """
    model = Sequential()
    
    # First layer
    model.add(Conv2D(16, (2, 2), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    ## Second layer
    #model.add(Conv2D(16, (2, 2), input_shape=input_shape))
    #model.add(Activation('relu'))
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    
    ## Third layer
    #model.add(Conv2D(8, (2, 2), input_shape=input_shape))
    #model.add(Activation('relu'))
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Final Layer
    model.add(Flatten())
    model.add(Dense(13))
    model.add(Activation('relu'))
    
    # Setup training mechanism
    model.compile(
        loss="mean_squared_error",
        optimizer=Adam(learning_rate=learning_rate))
    
    return model

## 3. Model Training

Finally, training the model using the single framed data and opening a tensorboard instance with details.

In [112]:
def train_model(run_name, tensorboard, model, xdata, ydata, models_folder="./models/"):
    """ Trains the given model with the given dataset. """
    model.fit(
        xdata,
        ydata,
        batch_size=2,
        validation_split=0.2,
        callbacks=[tensorboard],
        epochs=50
    )
    save_model(model, models_folder + run_name + ".mdl")

In [113]:
# Flexible parameters
learning_rate = 0.10

In [114]:
run_name = "single_frame-one_layer-lr_{}-{}".format(learning_rate, int(time.time()))
tb = TensorBoard(log_dir=".\\logs\\{}".format(run_name))

In [115]:
model = model_generator(learning_rate=learning_rate, input_shape=training_videos.shape[1:])

In [ ]:
train_model(run_name, tb, model, training_videos, training_rainfall)

Train on 143 samples, validate on 36 samples
Epoch 1/50
143/143 [==============================] - 2s 11ms/sample - loss: 14438.3003 - val_loss: 11153.4401
Epoch 2/50
143/143 [==============================] - 1s 7ms/sample - loss: 9380.0616 - val_loss: 10817.5255
Epoch 3/50
143/143 [==============================] - 1s 5ms/sample - loss: 9087.2428 - val_loss: 10521.0301
Epoch 4/50
143/143 [==============================] - 1s 6ms/sample - loss: 8829.6857 - val_loss: 10263.6337
Epoch 5/50
143/143 [==============================] - 1s 6ms/sample - loss: 8603.8420 - val_loss: 10028.2010
Epoch 6/50
143/143 [==============================] - 1s 5ms/sample - loss: 8408.2816 - val_loss: 9826.4771
Epoch 7/50
143/143 [==============================] - 1s 4ms/sample - loss: 8238.2187 - val_loss: 9653.1599
Epoch 8/50
143/143 [==============================] - 1s 4ms/sample - loss: 8092.3314 - val_loss: 9496.9363
Epoch 9/50
143/143 [==============================] - 1s 4ms/sample - loss: 7963.628